# Setting up a DELFT3D simulation

This is the outline of the dset.py script from /src/ folder that sets up a complete DELFT3D simulation

In [2]:
#python modules
import numpy as np
import datetime
import sys
import os
from shutil import copy2
import glob
import os
import xml.dom.minidom as md

In [3]:
#local modules
from meteo import wmap
from grid import *
from dep import *
from dem import readem
from idelft3d import meteo2delft3d
import mdf
from setobs import createf


ImportError: No module named meteo

## input variables

In [ ]:
# lat/lon window
lon0=-47.
lon1=44.
lat0=25.
lat1=76.

In [ ]:
nt=72 # number of forecasting intervals (hours)

In [ ]:
resolution = .1 # desired resolution in decimal degrees.

In [ ]:
#modify the lat/lon window to match the resolution 
ni=int((lon1-lon0)/resolution)+1
nj=int((lat1-lat0)/resolution)+1
  
lon1=lon0+ni*resolution
lat1=lat0+nj*resolution

In [ ]:
basename='test' # the name tag of the simulation. It will used in all the required files --see below.

In [ ]:
#This is the date stamp of the run. It will also be used the name of the simulation folder in YearMonthDay.Hour format.
rundate = datetime.date(2017,2,10,0)

In [ ]:
path = '../test/' # the location where the simulation will be setup

In [ ]:
# create the folder/run path
rpath=datetime.datetime.strftime(rundate,'%Y%m%d.%H' )

In [ ]:
calc_dir=path+'{}/'.format(rpath)  #run subfolder 
if not os.path.exists(rpath): #create folder if it doesn't exist
      os.makedirs(rpath)


## Create forcing

In [ ]:
#get meteo for that day... see Meteo.ipynb for more info
p,u,v,elat,elon = wmap(rundate,0,3*(nt+1),lon0,lon1,lat0,lat1)

In [ ]:
#write u,v,p files 
dlat=elat[1,0]-elat[0,0] #dlat required in meteo2delft
dlon=elon[0,1]-elon[0,0] #dlon required 
mlat0=elat[0,0] 
mlon0=elon[0,0] 


In [ ]:
meteo2delft3d(p,u,v,lat0,lon0,dlat,dlon,rundate,nt,path=rpath,curvi=False) # write to ascii delft3d format

##  Get bathymetry interpolated onto lon,lat

In [ ]:
# set the grid 
x=np.linspace(lon0,lon1,ni)
y=np.linspace(lat0,lat1,nj)
lon,lat=np.meshgrid(x,y)

In [ ]:
pathb='../BATHYMETRY/GLOBAL/GEBCO_2014_2D.nc' #global bathymetry file

In [ ]:
bat = readem(lat0,lat1,lon0,lon1,pathb,lon,lat,plot=False,interpolate=True)

In [ ]:
# Write bathymetry file
ba = Dep() # assign a bathymetry class variable
# append the line/column of nodata 
nodata=np.empty(ni)
nodata.fill(np.nan)
bat1=np.vstack((bat,nodata))
nodata=np.empty((nj+1,1))
nodata.fill(np.nan)
bat2=np.hstack((bat1,nodata))
ba.val = -bat2
ba.shape = bat2.shape

ba.val[ba.val<0]=-999.  # mask all dry points

Dep.write(ba,calc_dir+basename+'.dep') #write dep file

## Create the GRID file

In [ ]:
grd = Grid() #assign a grid class variable
  
grd.shape = bat.shape
grd.x = lon
grd.y = lat
grd.properties = {'Coordinate System': 'Spherical', 'alfori': 0.0, 'xori': 0.0, 'yori': 0.0}
  
grd.write(calc_dir+basename+'.grd') # write grd file

## Rest of the files

In [ ]:
# Write .enc file
  
with open(calc_dir+basename+'.enc','w') as f:
    f.write('{:>5}{:>5}\n'.format(ni+1,1))  # add one like ddb. This is the fortran/python conversion 0->1
    f.write('{:>5}{:>5}\n'.format(ni+1,nj+1))
    f.write('{:>5}{:>5}\n'.format(1,nj+1))
    f.write('{:>5}{:>5}\n'.format(1,1))
    f.write('{:>5}{:>5}\n'.format(ni+1,1))
  

In [ ]:
# Write .obs file. See Obs_points.ipynb
createf(calc_dir,basename,lat0,lat1,lon0,lon1,grd,ba)

In [ ]:
# Define the mdf input file
# first read the default
inp, order = mdf.read('default.mdf')

In [ ]:
# modify default mdf file

#Set grid file
inp['Filcco']=basename+'.grd'
  
#Set enc file
inp['Filgrd']=basename+'.enc'
  
#Set dep file
inp['Fildep']=basename+'.dep'
  
#Set obs file
inp['Filsta']=basename+'.obs'
  
# adjust ni,nj
inp['MNKmax']=[ni+1,nj+1,1]  # add one like ddb
  
# adjust iteration date
inp['Itdate']=datetime.datetime.strftime(runtime.date(),'%Y-%m-%d')
  
#set time unit
inp['Tunit']='M'

#adjust iteration start
inp['Tstart']=[Tstart]
  
#adjust iteration stop
inp['Tstop']=[Tstop]
  
#adjust time step
inp['Dt']=[1.]
  
#adjust time for output
step=60
inp['Flmap']=[Tstart,step,Tstop]
inp['Flhis']=[Tstart,1,Tstop]
inp['Flpp']=[0,0,0]
inp['Flrst']=[720]
  
#time interval to smooth the hydrodynamic boundary conditions
inp['Tlfsmo']=[0.]

# specify ini file
# if 'Filic' not in order: order.append('Filic')
# inp['Filic']=basename+'.ini'

# netCDF output
if 'FlNcdf' not in order: order.append('FlNcdf')
inp['FlNcdf'] = 'map his'

  
#SAVE mdf file
mdf.write(inp, calc_dir+basename+'.mdf',selection=order)


In [ ]:
# edit and save config files
#DELFT3D config xml file
copy2('config_d_hydro.xml',calc_dir+'config_d_hydro.xml')

xml=md.parse(calc_dir+'config_d_hydro.xml')

xml.getElementsByTagName('mdfFile')[0].firstChild.replaceWholeText(basename+'.mdf')

with open(calc_dir+'config_d_hydro.xml','w') as f:
      xml.writexml(f)

#execution shell script        
copy2('run_flow2d3d.sh',calc_dir+'run_flow2d3d.sh')